In [ ]:
# ! pip install pymupdf

In [11]:
import os
import warnings
warnings.filterwarnings("ignore")

In [12]:
from langchain_community.document_loaders import PyMuPDFLoader

# PyMuPDFLoader 을 이용해 PDF 파일 로드
loader = PyMuPDFLoader("C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/test/labor_low.pdf")
pages = loader.load()

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 문서를 문장으로 분리
## 청크 크기 500, 각 청크의 50자씩 겹치도록 청크를 나눈다
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
docs = text_splitter.split_documents(pages)

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    model_kwargs={'device':'cpu'},
    # model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True},
)

In [15]:
# 벡터 저장소 생성
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(docs, embeddings)


# 벡터 저장소 경로 설정
## 현재 경로에 'vectorstore' 경로 생성
vectorstore_path = 'vectorstore'
os.makedirs(vectorstore_path, exist_ok=True)

# 벡터 저장소 생성 및 저장
vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=vectorstore_path)
# 벡터스토어 데이터를 디스크에 저장
vectorstore.persist()
print("Vectorstore created and persisted")

Vectorstore created and persisted


In [16]:
from langchain_community.chat_models import ChatOllama

# Ollama 를 이용해 로컬에서 LLM 실행
## llama3-ko-instruct 모델 다운로드는 Ollama 사용법 참조
model = ChatOllama(model="llama3-ko", temperature=0)

In [17]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

In [26]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


# Prompt 템플릿 생성
template = '''당신은 구청에서 근무하는 공무원입니다. 귀하의 임무는 필요한 법률 및 근로에 관한 사실들에 대해 설명하는 것입니다.
최대한 자세하고 길게 설명해야하며 중요한 것은 이용자가 이해할 수 있는 쉽고 정확한 설명을 하는 것입니다. 효과적이고 자세하게 설명하세요.
실제 구청 직원이 잘 모르는 노인에게 대답하는 것 처럼 친절하고 다양한 예를 들어서 설명해야 합니다.

예를 들어 "각국의 법률에서는 정의라는 개념이 자주 등장하며, 법령의 형성과 해석에 있어 매우 중요한 부분을 차지한다. 
하지만 정의란 명확히 규정 할 수 없는 개념이기에 해석의 논란 이 있을 수 있다. 
그렇다면 사회구성원의 대다수가 납득할 수 있는 보편적 정의를 입증하는 방법은 무엇일지 생각해보아라" 와 같은 질문을 한다면,
"정의라는 개념은 법률에서 뿐만 아니라 윤리, 도덕, 정치 등 다양한 분야에서 일정한 의미를 지니고 있는 것으로, 
대다수 사람들이 공감하고 인정할 수 있는 보편적 정의를 규정하는 것은 어려운 과제입니다. 그러나 이를 위해 
다음과 같은 방법을 고려할 수 있습니다. 

첫째, 정의의 다양한 정의를 분석하여 공통점을 찾아내는 것입니다. 
예를 들어, 법률에서 정의는 일반적으로는 '법률에 의해 보호받을 권리'로 정의되지만, 윤리에서는 '도덕적으로 옳은 행동'으로, 
정치에서는 '국가의 이익'으로 정의됩니다. 이러한 다양한 정의를 분석하여 공통점을 찾아내면, 정의의 핵심요소를 찾아낼 수 있습니다. 

둘째, 정의의 역사적 발전을 연구하여 정의의 본질을 찾아내는 것입니다. 정의의 개념은 역사적으로 발전해 왔으며, 
다양한 문화와 사회에서 정의의 의미가 다르게 해석되었습니다. 이러한 역사적 발전을 연구하여 정의의 본질을 찾아내면, 
정의의 보편성을 확인할 수 있습니다. 

셋째, 정의의 다양한 측면을 고려하여 정의의 다원성을 인정하는 것입니다. 
정의는 법률, 윤리, 정치 등 다양한 분야에서 다르게 해석되며, 이러한 다원성을 인정하여 정의의 보편성을 확인할 수 있습니다. 
예를 들어, 법률에서는 정의는 권리 보호를 의미하지만, 윤리에서는 정의는 도덕적 가치 실현을 의미합니다. 
이러한 다양한 측면을 고려하여 정의의 다원성을 인정하면, 보편적 정의를 규정하는 데 도움이 됩니다. 

넷째, 사회 구성원의 의견을 수렴하여 정의의 보편성을 확인하는 것입니다. 정의는 사회 구성원들의 공감과 인정에 
의해 규정됩니다. 따라서, 사회 구성원의 의견을 수렴하여 정의의 보편성을 확인할 수 있습니다. 예를 들어, 설문 조사나 공청회를 
통해 사회 구성원들의 의견을 수렴하여 정의의 보편성을 확인할 수 있습니다. 

다섯째, 정의의 보편성을 확인하기 위해 국제적인 협력을 추진하는 것입니다. 
정의는 국가적 차원의 문제가 아니라 국제적인 문제입니다. 따라서, 국제적인 협력을 추진하여 정의의 보편성을 확인할 수 있습니다. 
예를 들어, 유엔 등 국제 기구와의 협력을 통해 정의의 보편성을 확인할 수 있습니다. 이러한 방법을 통해 보편적 정의를 규정할 수 있습니다. 
그러나 정의라는 개념은 매우 복잡하고 다양하므로, 보편적 정의를 규정하는 것은 쉽지는 않습니다. 하지만 이러한 방법을 통해 정의의 보편성을 확인하고, 
사회 구성원의 대다수가 납득할 수 있는 보편적 정의를 규정할 수 있습니다" 와 같이 대답할 수 있다.


답변은 한국어로만 답변해드립니다.":
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
query = "연장근로수당에 대해 자세히 알려줘"
answer = rag_chain.invoke(query)

print("Query:", query)
print("Answer:", answer)

Query: 연장근로수당에 대해 자세히 알려줘
Answer: 연장근로는 주 40시간의 근무를 넘어서는 근로입니다. 따라서, 사용자는 통상임금의 50% 이상을 가산하여 지급해야 합니다


In [28]:
import transformers
import torch

model_id = "Bllossom/llama-3-Korean-Bllossom-70B"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

pipeline.model.eval()

PROMPT = '''당신은 유용한 AI 어시스턴트입니다. 사용자의 질의에 대해 친절하고 정확하게 답변해야 합니다.
You are a helpful AI assistant, you'll need to answer users' queries in a friendly and accurate manner.'''
instruction = "서울과학기술대학교 MLP연구실에 대해 소개해줘"

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
    ]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print(outputs[0]["generated_text"][len(prompt):])

# 서울과학기술대학교 MLP연구실은 멀티모달 자연어처리 연구를 하고 있습니다. 구성원은 임경태 교수와 김민준, 김상민, 최창수, 원인호, 유한결, 임현석, 송승우, 육정훈, 신동재 학생이 있습니다.


KeyboardInterrupt: 

In [35]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

messages = [
    {"role": "system", "content": "친절한 챗봇으로서 상대방의 요청에 최대한 자세하고 친절하게 답하자. 모든 대답은 한국어(Korean)으로 대답해줘."},
    {"role": "user", "content": "피보나치 수열이 뭐야? 그리고 피보나치 수열에 대해 파이썬 코드를 짜줘볼래?"},
]

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=1,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

In [32]:
gen_text

'In a shocking finding, scientists discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. The study was published in the journal Scientific Reports. (Published February 11, 2016)\n\n[Related: Why Unicorns Are Surprising]\n\nUnicorns, as a herd, have been a curiosity throughout history, and in 2011, scientists were able to confirm'